In [1]:
# 2023-10-04
# Goal: Identify the ecDNA+ CCLE lines
#       Identify gene dependencies in these lines
# @author Owen Chapman

In [18]:
import pandas as pd
import os
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_classif
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:

# CCLE files. Obtain from https://ampliconrepository.org/project/65008f133d950b28f536202b
ecDNA_data_file = os.path.join("results/aggregated_results.csv")
# DepMap files. Obtain from https://depmap.org/portal/download/all/
dep_data_file = os.path.join("CRISPRGeneDependency.csv")
dep_name_file = os.path.join("Model.csv")

ecDNA_tbl = pd.read_csv(ecDNA_data_file,index_col=0)
dep_tbl = pd.read_csv(dep_data_file)
name_tbl = pd.read_csv(dep_name_file,index_col="ModelID")
dep_tbl

,ModelID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,0.055321,0.014039,0.014084,0.033828,0.049511,0.004955,0.026993,0.131006,0.003102,...,0.047757,0.047806,0.007155,0.002537,0.003660,0.154849,0.015792,0.015999,0.006200,0.436777
1,ACH-000004,0.023418,0.048724,0.058084,0.019483,0.049793,0.064472,0.001775,0.071289,0.003732,...,0.044517,0.471009,0.005096,0.015149,0.009804,0.015507,0.319598,0.007778,0.004392,0.048136
2,ACH-000005,0.059552,0.025478,0.009989,0.008775,0.099322,0.099572,0.007544,0.049601,0.047555,...,0.073457,0.281892,0.044789,0.058229,0.029638,0.097357,0.066269,0.025365,0.042530,0.096150
3,ACH-000007,0.023880,0.035082,0.006556,0.004322,0.022200,0.017121,0.009605,0.149378,0.052873,...,0.160608,0.593581,0.017412,0.001892,0.006432,0.029261,0.369101,0.015412,0.165622,0.366325
4,ACH-000009,0.027652,0.074860,0.011021,0.009153,0.021632,0.067758,0.013559,0.216796,0.013660,...,0.181632,0.342313,0.042414,0.002859,0.058070,0.053355,0.651365,0.012016,0.023990,0.312202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,ACH-002800,0.019522,0.048489,0.003258,0.008269,0.203445,0.010994,0.015949,0.047357,0.064302,...,0.006283,0.177775,0.024870,0.002609,0.028825,0.032515,0.155789,0.037765,0.034490,0.042029
1091,ACH-002834,0.035125,0.054841,0.015211,0.016620,0.154514,0.035249,0.011651,0.674236,0.013524,...,0.041955,0.716133,0.037090,0.042441,0.038526,0.020362,0.017330,0.224894,0.054621,0.562344
1092,ACH-002847,0.032003,0.045283,0.014737,0.010463,0.086174,0.017009,0.009417,0.405213,0.032471,...,0.261910,0.956598,0.008875,0.008511,0.015476,0.037763,0.145848,0.138299,0.053118,0.553382
1093,ACH-002922,0.024223,0.011662,0.012255,0.000397,0.135939,0.038571,0.004232,0.763590,0.007554,...,0.073170,0.899539,0.017797,0.001856,0.117058,0.034875,0.177452,0.056826,0.013545,0.295056


In [20]:
# get ecDNA+/- sets for CCLE.
def get_CCLE_ecDNA_y():
    return set(ecDNA_tbl[ecDNA_tbl.Classification == 'ecDNA']["Sample name"].drop_duplicates())
def get_CCLE_ecDNA_n():
    y = get_CCLE_ecDNA_y()
    return set(ecDNA_tbl["Sample name"])-y
dep_tbl

,ModelID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,0.055321,0.014039,0.014084,0.033828,0.049511,0.004955,0.026993,0.131006,0.003102,...,0.047757,0.047806,0.007155,0.002537,0.003660,0.154849,0.015792,0.015999,0.006200,0.436777
1,ACH-000004,0.023418,0.048724,0.058084,0.019483,0.049793,0.064472,0.001775,0.071289,0.003732,...,0.044517,0.471009,0.005096,0.015149,0.009804,0.015507,0.319598,0.007778,0.004392,0.048136
2,ACH-000005,0.059552,0.025478,0.009989,0.008775,0.099322,0.099572,0.007544,0.049601,0.047555,...,0.073457,0.281892,0.044789,0.058229,0.029638,0.097357,0.066269,0.025365,0.042530,0.096150
3,ACH-000007,0.023880,0.035082,0.006556,0.004322,0.022200,0.017121,0.009605,0.149378,0.052873,...,0.160608,0.593581,0.017412,0.001892,0.006432,0.029261,0.369101,0.015412,0.165622,0.366325
4,ACH-000009,0.027652,0.074860,0.011021,0.009153,0.021632,0.067758,0.013559,0.216796,0.013660,...,0.181632,0.342313,0.042414,0.002859,0.058070,0.053355,0.651365,0.012016,0.023990,0.312202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,ACH-002800,0.019522,0.048489,0.003258,0.008269,0.203445,0.010994,0.015949,0.047357,0.064302,...,0.006283,0.177775,0.024870,0.002609,0.028825,0.032515,0.155789,0.037765,0.034490,0.042029
1091,ACH-002834,0.035125,0.054841,0.015211,0.016620,0.154514,0.035249,0.011651,0.674236,0.013524,...,0.041955,0.716133,0.037090,0.042441,0.038526,0.020362,0.017330,0.224894,0.054621,0.562344
1092,ACH-002847,0.032003,0.045283,0.014737,0.010463,0.086174,0.017009,0.009417,0.405213,0.032471,...,0.261910,0.956598,0.008875,0.008511,0.015476,0.037763,0.145848,0.138299,0.053118,0.553382
1093,ACH-002922,0.024223,0.011662,0.012255,0.000397,0.135939,0.038571,0.004232,0.763590,0.007554,...,0.073170,0.899539,0.017797,0.001856,0.117058,0.034875,0.177452,0.056826,0.013545,0.295056


In [21]:
# map between DepMap and CCLE IDs
from itertools import chain
name_map = name_tbl["CCLEName"].dropna()
name_map = name_map[name_map.isin(get_CCLE_ecDNA_y()|get_CCLE_ecDNA_n())]
# print(name_map)
yes = get_CCLE_ecDNA_y() #ecDNA+ samples
no = get_CCLE_ecDNA_n() #ecDNA- samples
ecDNA_positive = list(yes)
ecDNA_negative = list(no)
dict_positive = {}
dict_negative = {}
for p in ecDNA_positive:
    dict_positive[p] = "ecDNA+" #dictionary that maps CCLE samples to ecDNA+
for p in ecDNA_negative:
    dict_negative[p] = "ecDNA-" #dictionary that maps CCLE samples to ecDNA-
dict_negative
ecDNA_status = dict(chain(dict_positive.items(), dict_negative.items())) #dictionary that combines CCLE samples and ecDNA status
name_map = name_map.to_dict()
# print(dep_tbl)
dep_tbl["CCLE_Name"] = dep_tbl["ModelID"].map(name_map) #maps DepMap MdoelIDs to CCLE names
dep_tbl

,ModelID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009),CCLE_Name
0,ACH-000001,0.055321,0.014039,0.014084,0.033828,0.049511,0.004955,0.026993,0.131006,0.003102,...,0.047806,0.007155,0.002537,0.003660,0.154849,0.015792,0.015999,0.006200,0.436777,NIHOVCAR3_OVARY
1,ACH-000004,0.023418,0.048724,0.058084,0.019483,0.049793,0.064472,0.001775,0.071289,0.003732,...,0.471009,0.005096,0.015149,0.009804,0.015507,0.319598,0.007778,0.004392,0.048136,NaN
2,ACH-000005,0.059552,0.025478,0.009989,0.008775,0.099322,0.099572,0.007544,0.049601,0.047555,...,0.281892,0.044789,0.058229,0.029638,0.097357,0.066269,0.025365,0.042530,0.096150,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
3,ACH-000007,0.023880,0.035082,0.006556,0.004322,0.022200,0.017121,0.009605,0.149378,0.052873,...,0.593581,0.017412,0.001892,0.006432,0.029261,0.369101,0.015412,0.165622,0.366325,LS513_LARGE_INTESTINE
4,ACH-000009,0.027652,0.074860,0.011021,0.009153,0.021632,0.067758,0.013559,0.216796,0.013660,...,0.342313,0.042414,0.002859,0.058070,0.053355,0.651365,0.012016,0.023990,0.312202,C2BBE1_LARGE_INTESTINE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,ACH-002800,0.019522,0.048489,0.003258,0.008269,0.203445,0.010994,0.015949,0.047357,0.064302,...,0.177775,0.024870,0.002609,0.028825,0.032515,0.155789,0.037765,0.034490,0.042029,NaN
1091,ACH-002834,0.035125,0.054841,0.015211,0.016620,0.154514,0.035249,0.011651,0.674236,0.013524,...,0.716133,0.037090,0.042441,0.038526,0.020362,0.017330,0.224894,0.054621,0.562344,NaN
1092,ACH-002847,0.032003,0.045283,0.014737,0.010463,0.086174,0.017009,0.009417,0.405213,0.032471,...,0.956598,0.008875,0.008511,0.015476,0.037763,0.145848,0.138299,0.053118,0.553382,NaN
1093,ACH-002922,0.024223,0.011662,0.012255,0.000397,0.135939,0.038571,0.004232,0.763590,0.007554,...,0.899539,0.017797,0.001856,0.117058,0.034875,0.177452,0.056826,0.013545,0.295056,NaN


In [22]:
dep_tbl["ecDNA Status"] = dep_tbl["CCLE_Name"].map(ecDNA_status) #maps depmap names to ecDNA status
dep_tbl.dropna(subset = ['ecDNA Status'], inplace=True)
# dep_tbl = dep_tbl.drop('ModelID', axis=1)
dep_tbl = dep_tbl.drop("CCLE_Name", axis = 1)
phenotype_labels = dep_tbl[["ModelID", "ecDNA Status"]]
dep_tbl

,ModelID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009),ecDNA Status
0,ACH-000001,0.055321,0.014039,0.014084,0.033828,0.049511,0.004955,0.026993,0.131006,0.003102,...,0.047806,0.007155,0.002537,0.003660,0.154849,0.015792,0.015999,0.006200,0.436777,ecDNA+
2,ACH-000005,0.059552,0.025478,0.009989,0.008775,0.099322,0.099572,0.007544,0.049601,0.047555,...,0.281892,0.044789,0.058229,0.029638,0.097357,0.066269,0.025365,0.042530,0.096150,ecDNA-
3,ACH-000007,0.023880,0.035082,0.006556,0.004322,0.022200,0.017121,0.009605,0.149378,0.052873,...,0.593581,0.017412,0.001892,0.006432,0.029261,0.369101,0.015412,0.165622,0.366325,ecDNA-
4,ACH-000009,0.027652,0.074860,0.011021,0.009153,0.021632,0.067758,0.013559,0.216796,0.013660,...,0.342313,0.042414,0.002859,0.058070,0.053355,0.651365,0.012016,0.023990,0.312202,ecDNA-
6,ACH-000012,0.054350,0.012480,0.007296,0.004775,0.017181,0.084226,0.002160,0.135486,0.019387,...,0.941892,0.005066,0.010182,0.025559,0.011567,0.036698,0.064189,0.037076,0.144897,ecDNA+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,ACH-001129,0.006154,0.032343,0.021700,0.005665,0.091387,0.016325,0.013166,0.005379,0.011031,...,0.218168,0.009847,0.044510,0.016679,0.007559,0.002623,0.006935,0.134144,0.304899,ecDNA-
695,ACH-001190,0.026710,0.019154,0.005128,0.026278,0.045072,0.132991,0.017355,0.230153,0.009186,...,0.470068,0.014659,0.010182,0.013974,0.061360,0.185657,0.018651,0.034371,0.070354,ecDNA+
708,ACH-001239,0.035318,0.050906,0.014478,0.002191,0.051635,0.099943,0.004557,0.064424,0.012399,...,0.900483,0.003860,0.007130,0.081788,0.005220,0.166237,0.010065,0.183644,0.292474,ecDNA-
723,ACH-001306,0.028519,0.019188,0.029996,0.012988,0.034118,0.035692,0.012886,0.115870,0.014186,...,0.696278,0.041220,0.028705,0.081262,0.014079,0.075115,0.059332,0.433872,0.305545,ecDNA+


In [23]:
phenotype_labels.to_csv("phenotype_labels.csv", index = False)
phenotype_labels.head()

,ModelID,ecDNA Status
0,ACH-000001,ecDNA+
2,ACH-000005,ecDNA-
3,ACH-000007,ecDNA-
4,ACH-000009,ecDNA-
6,ACH-000012,ecDNA+


In [24]:
# df = df.drop('column_name', axis=1)
expression_data = dep_tbl.drop('ecDNA Status', axis = 1)
# expression_data.to_csv("expression_data.csv", index = False)
gene_set = expression_data.columns
gene_set = gene_set.to_list()
gene_set = gene_set[1:]
gene_set =  pd.DataFrame(gene_set, columns=['Genes'])
gene_set.reset_index()
for index, row in gene_set.iterrows():
    string1 = row["Genes"]
    gene, _ = string1.split(" ")
    gene_set.loc[index, "Genes"] = gene
gene_set.to_csv("gene_set.csv")

In [25]:
# expression_data.to_csv("expression_data.csv", index = False)
# splits genes from its indices to match gene names to gene set names
changes = {}
columns = expression_data.columns.tolist()
# print(type(columns))
for col in columns: 
    gene = str(col).split(" ")
    changes[col] = gene[0]
expression_data.rename(columns = changes, inplace = True)
expression_data.head()
expression_data.to_csv("expression_data.csv", index = True)

In [39]:
# ecDNA+ is less than ecDNA-
import pandas as pd
from scipy import stats
from statsmodels.stats.multitest import multipletests
df = dep_tbl
groups = df.groupby('ecDNA Status')
ecDNA_positive_group = df[df['ecDNA Status'] == 'ecDNA+']
ecDNA_negative_group = df[df['ecDNA Status'] == 'ecDNA-']
# print(groups.head())
# Create an empty DataFrame to store t-test results
t_test_results = {}
ranksums_test_results = {}
# print(df)
for gene in df.columns[1:-1]:  
    # print(ecDNA_positive)
    ecDNA_positive = ecDNA_positive_group[gene]
    ecDNA_negative = ecDNA_negative_group[gene]



    t_statistic, p_value = stats.ttest_ind(ecDNA_positive, ecDNA_negative)
    

    t_test_results[gene] = {'t_statistic': t_statistic, 'p_value': p_value}
    stat, p_value = stats.ranksums(ecDNA_positive, ecDNA_negative, alternative = "less")
    ranksums_test_results[gene]={'ranksums_statistic':stat, 'p_value':p_value}


t_results_df = pd.DataFrame.from_dict(t_test_results, orient='index')

# Save the DataFrame to a TSV file
t_results_df.to_csv('t_test_results.tsv', sep='\t')
rr_results_df = pd.DataFrame.from_dict(ranksums_test_results, orient='index')
rr_results_df.to_csv('ranksums_test_results.tsv', sep='\t')
# _, t_results_df['p_value_corrected'], _, _ = multipletests(results_df['p_value'], method='fdr_bh')

# Save the DataFrame to a TSV file
t_results_df = t_results_df.dropna(subset=['p_value'])

# df.head()

/Users/rkenkre/miniconda3/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [35]:
df.head()

,ModelID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009),ecDNA Status
0,ACH-000001,0.055321,0.014039,0.014084,0.033828,0.049511,0.004955,0.026993,0.131006,0.003102,...,0.047806,0.007155,0.002537,0.003660,0.154849,0.015792,0.015999,0.006200,0.436777,ecDNA+
2,ACH-000005,0.059552,0.025478,0.009989,0.008775,0.099322,0.099572,0.007544,0.049601,0.047555,...,0.281892,0.044789,0.058229,0.029638,0.097357,0.066269,0.025365,0.042530,0.096150,ecDNA-
3,ACH-000007,0.023880,0.035082,0.006556,0.004322,0.022200,0.017121,0.009605,0.149378,0.052873,...,0.593581,0.017412,0.001892,0.006432,0.029261,0.369101,0.015412,0.165622,0.366325,ecDNA-
4,ACH-000009,0.027652,0.074860,0.011021,0.009153,0.021632,0.067758,0.013559,0.216796,0.013660,...,0.342313,0.042414,0.002859,0.058070,0.053355,0.651365,0.012016,0.023990,0.312202,ecDNA-
6,ACH-000012,0.054350,0.012480,0.007296,0.004775,0.017181,0.084226,0.002160,0.135486,0.019387,...,0.941892,0.005066,0.010182,0.025559,0.011567,0.036698,0.064189,0.037076,0.144897,ecDNA+


In [ ]:
t_results_df['P_Value_Corrected'] = multi.multipletests(t_results_df['p_value'], method='fdr_bh')[1]
# filtered_genes = t_results_df.loc[t_results_df['P_Value_Corrected'] <= 0.1]
# len(filtered_genes)
t_results_df = t_results_df.sort_values('P_Value_Corrected')
t_results_df.head(10)

In [ ]:
filtered_results = rr_results_df[~rr_results_df['p_value'].isna()]
# filtered_results['P_Value_Corrected'] = multi.multipletests(filtered_results['p_value'], method='fdr_bh')[1]
# # filtered_genes = t_results_df.loc[t_results_df['P_Value_Corrected'] <= 0.1]
# # len(filtered_genes)
# filtered_results = filtered_results.sort_values('P_Value_Corrected')
# filtered_results.head(30)

In [ ]:
#how many hits, boxplot, look up the genes 
#how might this relate to survival of cells containing ecDNA/ecDNA genesis? 
#known oncogenes, good hit, ds break repair, recombination
#separate into 2 sets, based on sign of statistic
#prioritize ecDNA+ vulnerabilities 
#immediately throw out negative ranksums statistic
#confirm whether 1 means completely essential or not essential at all, look at dep map docs
# filtered_genes = t_results_df.loc[t_results_df['P_Value_Corrected'] <= 0.1]
filtered_genes = filtered_results.loc[filtered_results['ranksums_statistic'] < 0]
# print(len(filtered_genes))
filtered_genes['P_Value_Corrected'] = multi.multipletests(filtered_genes['p_value'], method='fdr_bh')[1]
# print(filtered_genes.head(10))
filtered_genes = filtered_genes.loc[filtered_genes['P_Value_Corrected'] < 0.2]
# filtered_genes.head(20)
filtered_genes.to_csv('less_filtered_genes.csv', index=True)

In [ ]:
filtered_genes.reset_index()
# filtered_genes.boxplot(by = filtered_genes.index)

filtered_genes = filtered_genes.rename(columns={"index":"Gene"})
# filtered_genes = filtered_genes.loc[filtered_genes['ranksums_statistic'] > 0]
# filtered_genes.head(13)
SMAD4 = df[["SMAD4 (4089)", "ecDNA Status"]]
filtered_expression.head(10)
# plot(filtered_expression)
# sns.boxplot(data=filtered_expression,y='SKOR2 (652991)',hue='ecDNA Status')
SMAD4['ecDNA Status'] = SMAD4['ecDNA Status'].map({'ecDNA+': True, 'ecDNA-': False})
# sns.boxplot(data=filtered_expression,y='SKOR2 (652991)',hue='ecDNA Status')
sns.boxplot(x = SMAD4['ecDNA Status'], 
            y = SMAD4['SMAD4 (4089)'], 
            hue = SMAD4['ecDNA Status']).set(
    xlabel='ecDNA Status', 
    ylabel='CERES Dependency Scores'
)
plt.show()

In [ ]:
MDM4 = df[["MDM4 (4194)", "ecDNA Status"]]
filtered_expression.head(10)
# plot(filtered_expression)
# sns.boxplot(data=filtered_expression,y='SKOR2 (652991)',hue='ecDNA Status')
MDM4['ecDNA Status'] = MDM4['ecDNA Status'].map({'ecDNA+': True, 'ecDNA-': False})
# sns.boxplot(data=filtered_expression,y='SKOR2 (652991)',hue='ecDNA Status')
sns.boxplot(x = MDM4['ecDNA Status'], 
            y = MDM4['MDM4 (4194)'], 
            hue = MDM4['ecDNA Status']).set(
    xlabel='ecDNA Status', 
    ylabel='CERES Dependency Scores'
)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# filtered_expression.set_index("ecDNA Status", inplace = True)

In [ ]:
sns.boxplot(x = filtered_expression['ecDNA Status'], 
            y = filtered_expression['SKOR2 (652991)'], 
            hue = filtered_expression['ecDNA Status']).set(
    xlabel='ecDNA Status', 
    ylabel='CERES Dependency Scores'
)
plt.show()

In [ ]:
MDM2 = df[["MDM2 (4193)", "ecDNA Status"]]
sns.boxplot(x = MDM2['ecDNA Status'], 
            y = MDM2['MDM2 (4193)'], 
            hue = filtered_expression['ecDNA Status']).set(
    xlabel='ecDNA Status', 
    ylabel='CERES Dependency Scores'
)
plt.show()

In [ ]:
temp = dep_tbl.groupby("ecDNA Status")
# print(result_df)
result_df = temp.mean().reset_index()
result_df = result_df.set_index("ecDNA Status", inplace = False)
result_df = result_df.transpose()
result_df = result_df.rename(index={"ecDNA Status": "Gene"})
result_df

In [ ]:
import scipy.stats as ss
from scipy.stats import ranksums
dictionary_pvals = {}
statistic, pvalue = ranksums(result_df["ecDNA+"], result_df["ecDNA-"])
# print(pvalue)
for i, row in result_df.iterrows():
    statistic, pvalue = ss.ranksums([row["ecDNA+"]], [row["ecDNA-"]])
    dictionary_pvals[i] = pvalue
print(len(dictionary_pvals))


In [ ]:
#ecDNA+ is greater than ecDNA-, assume the opposite for now
import pandas as pd
from scipy import stats
from statsmodels.stats.multitest import multipletests
df = dep_tbl
groups = df.groupby('ecDNA Status')
t_test_results = {}
ranksums_test_results = {}
for gene in df.columns[:-1]:  
    ecDNA_positive = groups.get_group('ecDNA+')[gene]
    ecDNA_negative = groups.get_group('ecDNA-')[gene]

    t_statistic, p_value = stats.ttest_ind(ecDNA_positive, ecDNA_negative)
    

    t_test_results[gene] = {'t_statistic': t_statistic, 'p_value': p_value}
    stat, p_value = stats.ranksums(ecDNA_positive, ecDNA_negative, alternative = "greater")
    ranksums_test_results[gene]={'ranksums_statistic':stat, 'p_value':p_value}


t_results_df = pd.DataFrame.from_dict(t_test_results, orient='index')

# Save the DataFrame to a TSV file
t_results_df.to_csv('t_test_results.tsv', sep='\t')
rr_results_df = pd.DataFrame.from_dict(ranksums_test_results, orient='index')
rr_results_df.to_csv('ranksums_test_results.tsv', sep='\t')

t_results_df = t_results_df.dropna(subset=['p_value'])

In [ ]:
filtered_results = rr_results_df[~rr_results_df['p_value'].isna()]
filtered_results['P_Value_Corrected'] = multi.multipletests(filtered_results['p_value'], method='fdr_bh')[1]
# filtered_genes = t_results_df.loc[t_results_df['P_Value_Corrected'] <= 0.1]
# len(filtered_genes)
filtered_results = filtered_results.sort_values('P_Value_Corrected')
filtered_results.head(16)

In [ ]:
filtered_results = filtered_results
filtered_genes = filtered_results.loc[filtered_results['ranksums_statistic'] > 0]
# print(len(filtered_genes))
filtered_genes['P_Value_Corrected'] = multi.multipletests(filtered_genes['p_value'], method='fdr_bh')[1]
# print(filtered_genes.head(10))
filtered_genes = filtered_genes.loc[filtered_genes['P_Value_Corrected'] < 0.2]
# filtered_genes.head(20)
filtered_genes.to_csv('greater_filtered_genes.csv', index=True)

In [ ]:
def get_CCLE_dep_tbl():
    # Subset and reindex depmap dependency data to include only CCLE samples with WGS analyzed for ecDNA.
    ccle_dep_tbl = dep_tbl[dep_tbl.index.isin(name_map.index)]
    ccle_dep_tbl.index = (name_map[ccle_dep_tbl.index])
    return ccle_dep_tbl.dropna(axis=1)

# What features best distinguish ecDNA in this dataset?
def get_markers(features,labels,classifier=f_classif):
    model = SelectKBest(classifier, k=20)
    model = model.fit(features,labels)
    model = model.set_output(transform="pandas")
    markers = model.transform(features)
    # alternately, 
    # features.loc[:,model.get_support()]
    return markers

def plot(markers):
    dims = (16, 4)
    fig, ax = plt.subplots(figsize=dims)
    t = markers.copy()
    t["ecDNA"]=t.index.isin(get_CCLE_ecDNA_y())
    t = pd.melt(t,id_vars="ecDNA")
    t["variable"] = t["variable"].map(lambda x: x.split()[0])
    sns.boxplot(data=t, x='variable',y='value',hue='ecDNA')

In [ ]:
features = get_CCLE_dep_tbl()
labels = features.index.isin(get_CCLE_ecDNA_y())
fstat_markers = get_markers(features,labels,classifier=f_classif)
chi2_markers = get_markers(features,labels,classifier=chi2)

In [ ]:
plot(chi2_markers)

In [ ]:
# Hits
"""
ANKRD11 - neural chromatin remodeler
ASNS - Asparagine synthetase
BRAF - proto-oncogene, MAPK/ERK signalling
CDK8 - cell cycle regulator
DCLRE1B - telomere protection, double-strand break induction.
EIF2AK4 - ribosome regulator, stress response. interacts with GCN1
FOXA1 - pioneer differentiation TF
GATA3 - hematopoetic differentiation TF
GCN1 - ribosome regulator, stress response. interacts with EIF2AK4
MDM2 - p53 regulation, recurrent ecDNA amplification
MDM4 - p53 regulation, recurrent ecDNA amplification
PGK1 - glycolysis
PPM1D - p53 regulation, recurrent ecDNA amplification
WRN - helicase, Holliday junction resolution
ZFR - rna-binding protein

TODO:
generate p-values, multiple hypothesis correction.
Remove genes amplified on ecDNA.
Confirm CRISPR score interpretation
Find drug dependency data
Test drugs
"""

In [41]:
len(dep_tbl["ecDNA Status"] == "ecDNA+")

243